In [3]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer.noise import (
    NoiseModel,
    depolarizing_error
)
from scipy.optimize import curve_fit

In [4]:
def run_with_noise(circuit, noise_level):
    noise_model = NoiseModel()
    depol_error = depolarizing_error(noise_level, 1)
    noise_model.add_all_qubit_quantum_error(depol_error, ['cx'])
    simulator = Aer.get_backend('qasm_simulator')
    transpiled_circuit = transpile(circuit, simulator)
    result = simulator.run(transpiled_circuit, noise_model=noise_model).result()
    counts = result.get_counts()
    return counts

In [ ]:
def extrapolate_to_zero_noise(noise_levels, results):
    def fit_func(noise, a, b):
        return a + b * noise
    popt, _ = curve_fit(fit_func, noise_levels, results)
    zero_noise_estimate = fit_func(0, *popt)
    return zero_noise_estimate

In [ ]:
qc = QuantumCircuit(2, 2)
qc.h(0)
qc.cx(0, 1)
qc.measure([0, 1], [0, 1])

noise_levels = [0.01, 0.05, 0.1]
results = []

for noise in noise_levels:
    result = run_with_noise(qc, noise)
    results.append(result.get('00', 0))

In [ ]:
zero_noise_estimate = extrapolate_to_zero_noise(noise_levels, results)
print("Estimated zero noise result for '00' outcome:", zero_noise_estimate)